In [1]:
import pandas as pd
#import os
#pip install openpyxl
import numpy as np
#import scipy as sp


In [2]:
Data=pd.read_excel("DataCS-2018-2021.xlsx",
                   sheet_name=None,
                   usecols="B:P",
                   skipfooter=15,
                   header=7)



#LOADING SHEETS
Data18=Data['2018']
Data19=Data['2019']
Data20=Data['2020']
Data21=Data['2021']

#Data.keys()
#RENAME

# RENAME COLUMNS
new_columns = {0: 'Ma CK', 
               1: 'Ten Cong ty',
               2: 'San',
               3: 'Nhom nganh',
               4: 'Tong Tai san',
               5: 'Tong No phai tra',
               6: 'Von Chu so huu',
               7: 'No ngan han',
               8: 'No dai han',
               9: 'Tai san co dinh',
               10: 'Gia von hang ban',
               11: 'Loi nhuan truoc thue',
               12: 'Chi phi lai vay',
               13: 'Von hoa thi truong',
               14: 'ROA'               
              }

#RENAME and DROP ROWS

Data18=Data18.rename(columns={Data18.columns[i]: new_col for i, new_col in new_columns.items()})
Data19=Data19.rename(columns={Data19.columns[i]: new_col for i, new_col in new_columns.items()})
Data20=Data20.rename(columns={Data20.columns[i]: new_col for i, new_col in new_columns.items()})
Data21=Data21.rename(columns={Data21.columns[i]: new_col for i, new_col in new_columns.items()})

#DROP ROW 1-2
Data18.drop([0,1], axis=0, inplace=True)
Data19.drop([0,1], axis=0, inplace=True)
Data20.drop([0,1], axis=0, inplace=True)
Data21.drop([0,1], axis=0, inplace=True)

# LOC trong San Giao dich HOSE HNX UPCOM
San_GD = ['HOSE','HNX','UPCOM']
Data18 = Data18[Data18['San'].isin(San_GD)].sort_values(by=['Von hoa thi truong'], ascending=False)
Data19 = Data19[Data19['San'].isin(San_GD)].sort_values(by=['Von hoa thi truong'], ascending=False)
Data20 = Data20[Data20['San'].isin(San_GD)].sort_values(by=['Von hoa thi truong'], ascending=False)
Data21 = Data21[Data21['San'].isin(San_GD)].sort_values(by=['Von hoa thi truong'], ascending=False)



In [3]:
# Tong hop Du lieu tu Nam 2018 - 2021
Data18['Nam']=2018
Data19['Nam']=2019
Data20['Nam']=2020
Data21['Nam']=2021
# Ket hop GDP va COVID
DataGDP_COVID=pd.read_csv("DataGDP_COVID.csv")
DataNganh=pd.read_csv("Index_NhomNganh.csv")

# Data Tong hop
DataAll=pd.concat([Data18, Data19,Data20,Data21]).reset_index(drop=True)
DataAll=DataAll.merge(DataGDP_COVID, how='left', on='Nam')
DataAll=DataAll.merge(DataNganh, how='left', on='Nhom nganh')



In [4]:
#Don Vi la Unit: Million M
DataAll.iloc[:,4:13]=DataAll.iloc[:,4:13]//10**6 # Chia lay lam tron
#Von hoa thi truong, Unit: Ty dong, Billion  
DataAll['Von hoa thi truong']=DataAll['Von hoa thi truong'].apply(np.ceil)  #lam trong so tu nhien


In [5]:
# Tinh toan Nang cao
#GRWTH = MarketCap / Book value of Equity
DataAll['GRWTH']=DataAll['Von hoa thi truong']*1000/DataAll['Von Chu so huu']
# SIZE = natural logarithym LN(Total ASSETS)
DataAll['SIZE'] = np.log(DataAll['Tong Tai san'].astype('float'))
# EBIT = LNTT + Chi phi lai vay => PRO = EBIT/Total Assets ; RISK=Std(EBIT)
DataAll['EBIT']=DataAll['Loi nhuan truoc thue']+abs(DataAll['Chi phi lai vay'])
# Profitability PRO = EBIT/Tong Tai san
DataAll['PROF']=DataAll['EBIT']/DataAll['Tong Tai san']
# LIQD	Tổng tài sản ngắn hạn/ Nợ ngắn hạn
DataAll['LIQD']=DataAll['Tong Tai san']/DataAll['No ngan han']
# UNIQ	Giá vốn hàng bán/Tổng tài sản
DataAll['UNIQ']=abs(DataAll['Gia von hang ban'])/DataAll['Tong Tai san']
# Tính hữu hình	TANG	Tài sản cố định/ Tổng tài sản
DataAll['TANG']=DataAll['Tai san co dinh']/DataAll['Tong Tai san']
#BIEN PHU THUOC: STLEV, LTLEV, BLEV
DataAll['STLEV']=DataAll['No ngan han']/DataAll['Tong Tai san']
DataAll['LTLEV']=DataAll['No dai han']/DataAll['Tong Tai san']
DataAll['BLEV']=DataAll['Tong No phai tra']/DataAll['Tong Tai san']

In [6]:
Output=DataAll.iloc[:,[0,1,2,3,4,13,15,16,17,18,19,20,22,23,24,25,26,27,28]]
Output=Output.dropna().reset_index(drop=True)


In [7]:
#FILTER : Công ty có Total Asset >= 1000B and MarCap >= 1000B 
Output_2021=Output[Output['Nam'] == 2021].sort_values(by='Tong Tai san', ascending=False)
Top_Tickets=Output_2021.loc[(Output['Tong Tai san']>=1000*10**3) & (Output['Von hoa thi truong']>=1000)].reset_index(drop=True)
Top_Tickets=Top_Tickets.iloc[:,[0,1]]
Top_Tickets['ID_TICKET'] = np.arange(Top_Tickets.shape[0])
Top_Tickets=Top_Tickets.iloc[:,[2,0]]

Output=pd.merge(Output, Top_Tickets, on='Ma CK', how='inner').reset_index(drop=True)
Output


,Ma CK,Ten Cong ty,San,Nhom nganh,Tong Tai san,Von hoa thi truong,Nam,GDP,COVID,INDS,GRWTH,SIZE,PROF,LIQD,UNIQ,TANG,STLEV,LTLEV,BLEV,ID_TICKET
0,ACV,Cảng Hàng không VN,UPCOM,Công nghiệp L1,53624854,186158,2018,7.17,0,1,6.04678,17.797523,0.143837,6.978783,0.154697,0.344006,0.143291,0.282603,0.425895,8
1,ACV,Cảng Hàng không VN,UPCOM,Công nghiệp L1,58176195,186158,2019,7.02,0,1,5.064527,17.878987,0.176227,8.925203,0.154615,0.280963,0.112042,0.256132,0.368174,8
2,ACV,Cảng Hàng không VN,UPCOM,Công nghiệp L1,56902290,186158,2020,2.91,1,1,4.955595,17.856846,0.036844,14.618409,0.112088,0.251948,0.068407,0.271423,0.339829,8
3,ACV,Cảng Hàng không VN,UPCOM,Công nghiệp L1,54979483,186158,2021,2.58,1,1,4.944082,17.822471,0.019627,14.907657,0.100717,0.227536,0.06708,0.248071,0.31515,8
4,MSN,Tập đoàn Masan,HOSE,Hàng Tiêu dùng L1,64578613,152624,2018,7.17,0,2,4.478446,17.983394,0.137992,4.088415,0.407352,0.45222,0.244594,0.227682,0.472276,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,SCG,Xây dựng SCG,HNX,Công nghiệp L1,1344047,6010,2020,2.91,1,1,10.091478,14.111196,0.083881,1.797018,0.951255,0.002972,0.556478,0.000418,0.556896,102
909,SCG,Xây dựng SCG,HNX,Công nghiệp L1,4871164,6010,2021,2.58,1,1,5.382758,15.398843,0.072369,2.255087,0.536786,0.00203,0.443442,0.327346,0.770788,102
910,TOS,Dịch vụ biển Tân Cảng,UPCOM,Dầu khí L1,2020004,1437,2019,7.02,0,6,1.844976,14.518610,0.110343,3.561669,0.448791,0.592188,0.280767,0.333653,0.614421,170
911,TOS,Dịch vụ biển Tân Cảng,UPCOM,Dầu khí L1,1821188,1437,2020,2.91,1,6,1.694575,14.415000,0.139237,3.891768,0.546628,0.525765,0.256953,0.277417,0.534369,170


In [8]:
Top_Tickets


,ID_TICKET,Ma CK
0,0,HPG
1,1,MSN
2,2,GVR
3,3,BSR
4,4,PLX
...,...,...
221,221,VGR
222,222,DSP
223,223,CQN
224,224,GVT


In [9]:
# EXPORT FILE DATA
with pd.ExcelWriter('Report.xlsx') as writer:
    DataAll.to_excel(writer, sheet_name='Data')
    Output.to_excel(writer, sheet_name='Output')
    Top_Tickets.to_excel(writer,sheet_name='Top Tickets')